# n-gram Model

### Step 1: Download and Load the Dataset

In [1]:
import requests
import os
from collections import Counter, defaultdict
import random

# Download Shakespeare dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

# Save the file
with open("shakespeare.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

print("Dataset downloaded and saved as 'shakespeare.txt'.")

Dataset downloaded and saved as 'shakespeare.txt'.


### Step 2: Load and Preprocess the Data

In [2]:
with open("shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Remove unnecessary whitespace and newlines
data = text.replace("\n", " ")

print(f"Dataset length: {len(data)} characters")

Dataset length: 1115394 characters


### Step 3: Build n-gram Model (n=2 and n=3)

+ Show how many distinct tuples are there in the training data

In [3]:
def generate_ngrams(text, n):
    """Generate n-grams from the given text."""
    ngrams = [tuple(text[i : i + n]) for i in range(len(text) - n)]
    return Counter(ngrams)

# Generate bigrams (n=2) and trigrams (n=3)
bigrams = generate_ngrams(data, 2)
trigrams = generate_ngrams(data, 3)

print(f"Total unique bigrams: {len(bigrams)}")
print(f"Total unique trigrams: {len(trigrams)}")

Total unique bigrams: 1318
Total unique trigrams: 10033


### Step 4: Find the Most Frequent n-grams

In [4]:
def most_frequent_ngrams(ngram_counter, top_n=1):
    """Return the most common n-grams."""
    return ngram_counter.most_common(top_n)

most_common_bigrams = most_frequent_ngrams(bigrams, 1)
most_common_trigrams = most_frequent_ngrams(trigrams, 1)

print(f"Most common bigram: {most_common_bigrams}")
print(f"Most common trigram: {most_common_trigrams}")

Most common bigram: [(('e', ' '), 29077)]
Most common trigram: [((' ', 't', 'h'), 16237)]


### Step 5: Find the Most Three Likely Next Character for Each n-gram

In [10]:
def compute_next_char_probabilities(text, n):
    """Compute the probability distribution of the next character given an n-gram prefix."""
    ngram_dict = defaultdict(Counter)
    for i in range(len(text) - (n-1)):
        prefix = tuple(text[i : i + n - 1])  # (xt-1, xt-2, ..., xt-n+1)
        next_char = text[i + n - 1]  # xt
        ngram_dict[prefix][next_char] += 1
    
    # Convert counts to probabilities
    for prefix, counter in ngram_dict.items():
        total_count = sum(counter.values())
        for char in counter:
            counter[char] /= total_count
    
    return ngram_dict

# Compute next character probabilities for bigrams and trigrams
bigram_next_char_probs = compute_next_char_probabilities(data, 2)
trigram_next_char_probs = compute_next_char_probabilities(data, 3)

# Show example output
example_bigram_prefix = most_common_bigrams[0][0][:-1]   # Most common bigram prefix
example_trigram_prefix = most_common_trigrams[0][0][:-1]   # Most common trigram prefix

top_3_bigram_chars = bigram_next_char_probs[example_bigram_prefix].most_common(3)
top_3_trigram_chars = trigram_next_char_probs[example_trigram_prefix].most_common(3)

print(f"Top 3 most likely next characters for bigram {example_bigram_prefix}: {top_3_bigram_chars}")
print(f"Top 3 most likely next characters for trigram {example_trigram_prefix}: {top_3_trigram_chars}")

Top 3 most likely next characters for bigram ('e',): [(' ', 0.3073321283994462), ('r', 0.1244147086491), ('n', 0.07999069875595861)]
Top 3 most likely next characters for trigram (' ', 't'): [('h', 0.672813160402768), ('o', 0.20453321178469316), ('r', 0.03488998466829652)]


### Step 6: Generate Text using the n-gram Model

In [11]:
def generate_text(ngram_dict, seed, length=100):
    """Generate text using an n-gram probability distribution."""
    generated = list(seed)
    for _ in range(length):
        prefix = tuple(generated[-(len(seed)):])  # Match the prefix length
        if prefix in ngram_dict:
            next_char = random.choices(
                list(ngram_dict[prefix].keys()), 
                weights=ngram_dict[prefix].values()
            )[0]
            generated.append(next_char)
        else:
            break  # Stop if no continuation found
    return ''.join(generated)

# Generate three paragraphs of text using bigrams and trigrams
bigram_seed = random.choice(list(bigram_next_char_probs.keys()))
trigram_seed = random.choice(list(trigram_next_char_probs.keys()))

print("\nGenerated Text with Bigrams:")
print(generate_text(bigram_next_char_probs, bigram_seed, length=200))
print(generate_text(bigram_next_char_probs, bigram_seed, length=200))
print(generate_text(bigram_next_char_probs, bigram_seed, length=200))

print("\nGenerated Text with Trigrams:")
print(generate_text(trigram_next_char_probs, trigram_seed, length=200))
print(generate_text(trigram_next_char_probs, trigram_seed, length=200))
print(generate_text(trigram_next_char_probs, trigram_seed, length=200))


Generated Text with Bigrams:
t dorday s: the cley tigano hive  I A: alun pite, hyou. s? for suremouour Isou; l Thoucu led h nstha w alouthos blla Are doulisod ms n. BUnd farg ay yong S: Iffay ire tisathavity conon pluthe skig thob
the te Fow AUCETARO: fag wer, w, s g's: he shototorag d I for isoligms ane th TIng gee d ar, outotou curvee fany atsshitajelyopr, l. te by athafrty w nst bie the HOFRI ce: co lloordinired ke t pousatou
thif y! Bof Th. tthy th s ind fo st ch y aist. Mat GLIsit whese arant Y a areff. t wan, ifur thown t Tofainpeak! hes, pp, thisucase, an; tal, timo llind stind fa hathele t testhege e d toman oches  The

Generated Text with Trigrams:
MPSABET: I a rearms, I to lions! wan th ented to You and hou love ind is, weet quare dot apprisladed yousantire so ase: And forbecompait fou anseelf himbe pre wilt ne: blord: Aband I seentills take heed
MPSABET: Not firm That Wary min and to mor tow on, sile havest.  PER: All not? O, him deam an, in eavers, wity, so und annamearwit

# Warmup for neural network and deep learning

### Step 1: Load and Preprocess Data

In [12]:
### Character Level Text Generation

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import Counter


In [13]:

# Load Shakespeare dataset
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Create character vocabulary
chars = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(chars)}
idx2char = {idx: char for char, idx in char2idx.items()}

# Convert text to indices
encoded_text = [char2idx[char] for char in text]


### Step 2: Define GRU-based Model

In [14]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.gru(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

### Step 3: Prepare Data for Training

In [15]:

def create_sequences(encoded_text, seq_length):
    sequences = []
    targets = []
    for i in range(len(encoded_text) - seq_length):
        sequences.append(encoded_text[i:i+seq_length])
        targets.append(encoded_text[i+1:i+seq_length+1])
    return torch.tensor(sequences), torch.tensor(targets)

seq_length = 100
sequences, targets = create_sequences(encoded_text, seq_length)


### Step 4: Train the Model

In [16]:
def train_model(model, data, targets, num_epochs=10, batch_size=64, learning_rate=0.003):
    """
    Train the GRU model and use CrossEntropyLoss as the loss function.
    """

    # Setting the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0

        for i in range(0, len(data), batch_size):
            inputs = data[i:i+batch_size].to(device)   # Make sure it's on the right device
            labels = targets[i:i+batch_size].to(device)

            # Make sure the hidden batch_size is correct.
            current_batch_size = inputs.shape[0]
            hidden = model.init_hidden(current_batch_size).to(device)  

            optimizer.zero_grad()  # Empty the gradient
            
            # forward propagation
            output, hidden = model(inputs, hidden)

            # Calculate loss
            loss = criterion(output.view(-1, len(chars)), labels.view(-1))
            
            # Reverse Spread & Update Parameters
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data):.4f}')



# Initialising the model and training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUModel(len(chars), embed_size=128, hidden_size=256, num_layers=2).to(device)
train_model(model, sequences, targets)


Epoch 1/10, Loss: 0.0296
Epoch 2/10, Loss: 0.0297
Epoch 3/10, Loss: 0.0302
Epoch 4/10, Loss: 0.0307
Epoch 5/10, Loss: 0.0305
Epoch 6/10, Loss: 0.0307
Epoch 7/10, Loss: 0.0319
Epoch 8/10, Loss: 0.0342
Epoch 9/10, Loss: 0.0341
Epoch 10/10, Loss: 0.0317


### Step 5: Generate Text

In [18]:
def generate_text(model, start_str, length=200):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialise the hidden and make sure it is on the correct device
    hidden = model.init_hidden(batch_size=1).to(device)

    # Convert start_str to index to make sure input_seq is on the right device
    input_seq = torch.tensor([char2idx[char] for char in start_str], dtype=torch.long).unsqueeze(0).to(device)
    generated_text = start_str

    for _ in range(length):
        output, hidden = model(input_seq, hidden)

        # Get the output of the last time step and convert it to a chance
        probs = torch.nn.functional.softmax(output[:, -1, :], dim=-1).detach().cpu().numpy()
        next_char_idx = np.random.choice(len(chars), p=probs.flatten())

        # Update Generated Text
        generated_text += idx2char[next_char_idx]

        # Update input_seq to make sure it's on the GPU
        input_seq = torch.tensor([[next_char_idx]], dtype=torch.long).to(device)

    return generated_text

# Generate and print example text
print(generate_text(model, 'To be or not to be', length=200))
print("\n")
print(generate_text(model, 'I', length=200))
print("\n")
print(generate_text(model, 'and', length=200))


To be or not to bead a didmy and amp in Datoubth baspes? Wray. Whiss apre have and spit? bet knon Drsysich so Mays as ay? I at awere rocpocaltempite daughan she as id and mness king-not, I shasta;
AThout they;
Thy stak


I at they plast makaning these of not youk,-Dard, such by lats maulible all nal?

ANTONIO:
Why dannt no bis me Dhere? -not pay? and you? spir? Saper ats and, arpe shall buph spit not didank all.

SEBAS


and leth enget ave? the daugcess eme these mink phereinconst soppfar a that atting appets?

SEBASTIAN:
Whe then onlencice,--a neve? add--- and nentiugle.

SEBASTIAN:
No take was's teedd.

ANTONIO:
That a
